# Welcome to nextnanopy !

## About this example: load and execute an input file

To work with an input file, simply do:

```python
my_input = nextnanopy.InputFile(path_to_file)
```

This object contains all the relevant information of the input file such as the raw text, the variables and the configuration for the execution (see Example 0).

When you create ```my_input```, the class will first check automatically the type of the input file (nextnano++, nextnano3, nextnano.NEGF or nexntano.MSB). Then, it will get the raw text and extract all variables defined in the file like:

```
$my_var = 3 # in nextnano++
%my_var = 3 ! in nextnano3
```

Note that loading variables or sweeping parameters are not implemented for nextnano.NEGF nor nextnano.MSB yet.

In [1]:
import nextnanopy as nn
my_input = nn.InputFile(r'E:\junliang.wang\datafiles\nextnano++\example.in')

## What information does it contain?

```my_input``` has the following relevant attributes:

- fullpath (str): path to the input file including the name
- text (str): return raw text
- variables (list): list of InputVariable objects
- config: nextnano configuration (see Example 0)


In [2]:
print(my_input.fullpath)

E:\junliang.wang\datafiles\nextnano++\example.in


In [3]:
print(my_input.text)

# 1D sample for solving quantum-poisson

#Begin NextnanoWizard parameter section
$BIAS = 0.0 # Gate voltage(V)
$ALLOY = 0.3 # Al content of AlGaAs layer
$SIZE = 80 # size of AlGaAs layer (nm)
$DOPECONC = 3e+18 # donor concentration of 10nm doping layer (cm^-3)
$DOPEPOS = 65 # distance between AlGaAs-GaAs interface and delta doping position (nm)
$MINGRID = 0.1 # minimum grid spacing (nm)
$NUMEV = 10 # number of eigenvalues to be calculated
$BARRIER = 0.7 # height of left Schottky barrier (eV)
#End NextnanoWizard parameter section


global{
   simulate1D{}
   
   temperature = 4

   substrate{ name = "GaAs" }

   crystal_zb{ x_hkl = [1, 0, 0] y_hkl = [0, 1, 0] }
}


grid{
   xgrid{
      line{ pos = 0                   spacing = 0.5 }
      line{ pos = 10                  spacing = 0.5 }
      line{ pos = 10+$SIZE-$DOPEPOS-5 spacing = 0.5 }
      line{ pos = 10+$SIZE-$DOPEPOS+5 spacing = 0.5 }
      line{ pos = 10+$SIZE            spacing = $MINGRID }
      line{ pos = 400               

## Input variables

The input variables are stored in ```my_input.variables``` as a list of ```InputVariable``` objects.

These objects have the following attributes:

- name (str)
- value (undefined)
- unit (str, optional)
- label (str, optional)
- comment (str, optional)
- metadata (dict, optional)
- text (str): raw text in the input file for a given variable

In [4]:
print(f"List of variables: {my_input.variables}")

List of variables: DictList([
(0,'BIAS',InputVariable_nnp("BIAS",...)),
(1,'ALLOY',InputVariable_nnp("ALLOY",...)),
(2,'SIZE',InputVariable_nnp("SIZE",...)),
(3,'DOPECONC',InputVariable_nnp("DOPECONC",...)),
(4,'DOPEPOS',InputVariable_nnp("DOPEPOS",...)),
(5,'MINGRID',InputVariable_nnp("MINGRID",...)),
(6,'NUMEV',InputVariable_nnp("NUMEV",...)),
(7,'BARRIER',InputVariable_nnp("BARRIER",...))
])


### Get a variables

There are several methods to do so:

```python
my_input.get_variable(name) # method 1 (recommended)
my_input.variables[name] # method 2 (recommended)
my_input.variables[index] # method 3
```

### Let's have a quick look of the variables in this file

In [5]:
for key, var in my_input.variables.items():
    print(var.text)

$BIAS = 0.0 # Gate voltage(V)
$ALLOY = 0.3 # Al content of AlGaAs layer
$SIZE = 80 # size of AlGaAs layer (nm)
$DOPECONC = 3e+18 # donor concentration of 10nm doping layer (cm^-3)
$DOPEPOS = 65 # distance between AlGaAs-GaAs interface and delta doping position (nm)
$MINGRID = 0.1 # minimum grid spacing (nm)
$NUMEV = 10 # number of eigenvalues to be calculated
$BARRIER = 0.7 # height of left Schottky barrier (eV)


### Variable line number

It is an information that you won't probably need, but you can see the line number in the input file where the variable is defined:

```python
my_input.variables['BIAS'].metadata
```

In [6]:
my_input.variables['BIAS'].metadata

{'line_idx': 3}

### Modify the variable

If you want to modify the value of a given variable, you can do:

```python
my_input.set_variable('BIAS',value=3)
```

You can also change the comment:
```python
my_input.set_variable('BIAS',comment='this is a new comment')
```

You can change both at the same time:
```python
my_input.set_variable('BIAS',value=3,comment='this is a new comment')
```

In [7]:
print(f"Old variable: {my_input.get_variable('BIAS').text}")
my_input.set_variable('BIAS',value=3,comment='this is a new comment')
print(f"New variable: {my_input.get_variable('BIAS').text}")

Old variable: $BIAS = 0.0 # Gate voltage(V)
New variable: $BIAS = 3 # this is a new comment


## Save the current file

```my_input``` has a method ```save``` which allows you to save the current file.

```python
my_input.save()
```

Note that by default, it won't overwrite the current file such that it will find an unused name (like example_1.in) and save it.

You can overwrite the current file:

```python
my_input.save(overwrite=True)
```

You can save it in another location and with another name:

```python
my_input.save('new_path\new_input.in')
```

Automatically, ```my_input.fullpath``` is updated if the saving was successful.

By default, if the new location does not exist, it will create the folder automatically. In order to turn this option off, you can do:

```python
my_input.save('new_path\new_input.in',automkdir=False)
```

In [8]:
help(my_input.save)

Help on method save in module nextnanopy.inputs:

save(fullpath=None, overwrite=False, automkdir=True) method of nextnanopy.inputs.InputFile instance



## Execute the input file

In order to execute the input file:

```python
my_input.execute()
```

Note that if you have modified one or more variables, you must save the file before execution. 

The ```execute``` method without any arguments, it will use the configuration stored in ```my_input.config```. As explained in Example 0, it will automatically detect the nextnano product (nextnano++, nextnano3, nextnano.NEGF or nextnano.MSB) and will load the corresponding configuration parameters prior execution.

The optional arguments are:

- exe: path to nextnano executable
- license: path to nextnano license file
- database: path to nextnano database file
- outputdirectory: path to output folder
- Other optional arguments depending of the nextnano product

In [9]:
my_input.execute()

C:\Program Files\nextnano\2020_10_16\nextnano++\bin 64bit
STARTING...
Starting execution as:
C:\Program Files\nextnano\2020_10_16\nextnano++\bin 64bit\nextnano++_Intel_64bit.exe --license C:\Users\jun-liang.wang\Documents\nextnano\License\License_nnp.lic --database C:\Program Files\nextnano\2020_10_16\nextnano++\Syntax\database_nnp.in --threads 0 --outputdirectory E:\junliang.wang\nextnano\Output\example --noautooutdir E:\junliang.wang\datafiles\nextnano++\example.in 

nextnano++ (1.6.79 - 2020.092201) Oct 16 2020
COPYRIGHT NOTICE                                                             
Please read the file 'copyright_nextnano++.txt' in your installation folder  
for further information about the terms of copyright of the nextnano++ code  
and of third party source codes and libraries used in the nextnano++ code.   
                                                                             
In case this file is missing or seems incomplete or corrupted, please contact
nextnano Gmb


Solving Quantum Mechanics ---- (and calculate density)

  valence band maximum: -1.51789155
  conduction band minimum: -0.0388656864

 Solving 1 approximate subspace 1-band Schroedinger equation(s)...
   Solving dense hermitian eigenvalue problem (standard solver)...

Computing densities...

QUANTUM-POISSON:   Residual_EDensity = 3.253110567e+06   Residual_HDensity = 0.000000000e+00
QUANTUM-POISSON:   Residual_potential = 3.656992953e-07

QUANTUM-POISSON:  iteration = 8 of 30 ----------------------------

Solving nonlinear Poisson equation...
  Newton step: 1	8.880327249267377e-08
  Newton achieved/desired residual: 5.53995026e-13 1.80951265e-10

Solving Quantum Mechanics ---- (and calculate density)

  valence band maximum: -1.51789155
  conduction band minimum: -0.0388656276

 Solving 1 exact 1-band Schroedinger equation(s)...
    Tridiagonal real symmetric eigenvalue solver:           1          10

Computing densities...

QUANTUM-POISSON:   Residual_EDensity = 1.978813726e+06   Re

## Sweep an input variable

Since you can change the input variable dynamically, you can sweep its value and execute the file in a simple loop!

```python
for value in [0, 1, 2]:
    my_input.set_variable('BIAS',value=value)
    my_input.save() # remember that by default, overwrite is False.
    my_input.execute() 
```

Please, contact python@nextnano.com for any issue with this example.